In [11]:
pip install chromadb

In [12]:
import chromadb
from chromadb.utils import embedding_functions

In [13]:
import json
from pathlib import Path

import chromadb
from chromadb.utils import embedding_functions


def preprocess_transactions(data):
    texts = []
    for t in data:
        text = f"On {t['date']}, {t['customer']} purchased a {t['product']} for {t['amount']}."
        texts.append(text)
    return texts

def load_or_create_transactions():
    sample_data = [
        {"id": 1, "customer": "Amit", "product": "Laptop", "amount": 55000, "date": "2024-01-12"},
        {"id": 2, "customer": "Amit", "product": "Mouse", "amount": 700, "date": "2024-02-15"},
        {"id": 3, "customer": "Riya", "product": "Mobile", "amount": 30000, "date": "2024-01-05"},
        {"id": 4, "customer": "Riya", "product": "Earbuds", "amount": 1500, "date": "2024-02-20"},
        {"id": 5, "customer": "Karan", "product": "Keyboard", "amount": 1200, "date": "2024-03-01"}
    ]

    path = Path("transactions.json")

    if not path.exists():
        path.write_text(json.dumps(sample_data, indent=2))
        print("transactions.json file created with sample data")

    return json.loads(path.read_text())

transactions = load_or_create_transactions()


documents = preprocess_transactions(transactions)

metadatas = []
ids = []
for t in transactions:
        ids.append(str(t["id"]))

        metadatas.append({
            "customer": t["customer"],
            "product": t["product"],
            "amount": t["amount"],
            "date": t["date"],
            "source": "transactions.json"
        })

chroma_client = chromadb.PersistentClient(path="my_vectordb")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
model_name="all-mpnet-base-v2")


collection = chroma_client.get_or_create_collection(
        name="my_collection",
        embedding_function=sentence_transformer_ef
    )

collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )

print(f"Inserted {len(documents)} documents into collection 'my_collection'")



Inserted 5 documents into collection 'my_collection'


In [14]:

chroma_client = chromadb.PersistentClient(path="my_vectordb")



sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")


collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef)




collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids

)

In [15]:
pip install groq


In [16]:
import groq
from groq import Groq
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.PersistentClient(path="my_vectordb")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef)

client = Groq(api_key="---")


user_query = input("User: ")
query = user_query


results = collection.query(
    query_texts=[query],
    n_results=2,
    include=["documents", "metadatas", "distances"]
)


context_chunks = results["documents"][0]
context = "\n".join(context_chunks)


rag_prompt = f"""
You are a RAG chatbot. Use ONLY the context below to answer the question.

Context:
{context}

Question:
{query}

Answer:
"""

messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": rag_prompt}
]


try:
    response = client.chat.completions.create(
        messages=messages,
        model="llama-3.1-8b-instant"
    )

    answer = response.choices[0].message.content
    print("\nAI Assistant:", answer)

except groq.APIConnectionError as e:
    print("Cannot reach Groq server.")
    print(e._cause_)

except groq.RateLimitError:
    print("Rate limit exceeded.")

except groq.APIStatusError as e:
    print("API Error:", e.status_code)
    print(e.response)

User:  Show me Riya’s purchase history.

AI Assistant: Date | Item | Price
-----|------|------
2024-01-05 | Mobile | 30000
2024-02-20 | Earbuds | 1500
